# Privacy-Preserving CNN

## Preparations

### options

In [1]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### modules

In [2]:
import torch  # pytorch
import numpy as np  # handling vectors and matrices
from keras.models import load_model  # reload pretrained tf models
from analysis_functions import (train_tf_model, pred_tf_model,
                                train_pt_model, pred_pt_model)  # own functions
from preproc_functions import list_combine  # own functions
from neural_nets import TFUNET, TFMNET, PTUNET, PTMNET  # own defined neural nets as classes

# pysyft
import syft as sy
from syft.frameworks.torch.tensors.interpreters import PointerTensor
from syft.frameworks.torch.tensors.decorators import LoggingTensor
hook = sy.TorchHook(torch)  # hooking pysyft into pytorch
import torch.nn.functional as F
import torch.nn as nn

Using TensorFlow backend.


### datasets

#### univariate training

In [3]:
train_Xu = np.load(dir_data+'train_Xu.npy')
train_yu = np.load(dir_data+'train_yu.npy')

In [4]:
print(train_Xu.shape)
print(train_yu.shape)

(988, 14, 1)
(988, 7)


995 Observations for X days for 1 Variable!

#### multivariate training

In [5]:
train_Xm = np.load(dir_data+'train_Xm.npy')
train_ym = np.load(dir_data+'train_ym.npy')

In [6]:
print(train_Xm.shape)
print(train_ym.shape)

(988, 14, 8)
(988, 7)


995 Observations for X days for 8 Variables!

#### test set

In [7]:
test_Xu = np.load(dir_data+'test_Xs.npy')[:,:,:1]
test_Xm = np.load(dir_data+'test_Xs.npy')
test_y = np.load(dir_data+'test_ys.npy')

In [8]:
print(test_Xu.shape)
print(test_Xm.shape)
print(test_y.shape)

(59, 14, 1)
(59, 14, 8)
(59, 7)


## Models

### Options

In [9]:
# net hyper parameters
# general
lr = 0.001
n_filters = 16
n_linear = 10
# univariate specific
epochs_u = 20
batch_size_u = 4
# multivariate specific
epochs_m = 70
batch_size_m = 16

In [10]:
# dimensions
n_timesteps = train_Xu.shape[1] 
n_features_u = train_Xu.shape[2]
n_features_m = train_Xm.shape[2]
n_outputs = train_yu.shape[1] 

### load pretrained weights

#### tensorflow

In [11]:
# create net objects
model_tfu = TFUNET(n_features_u, n_filters, n_timesteps, n_linear, n_outputs)
model_tfm = TFMNET(n_features_m, n_filters, n_timesteps, n_linear, n_outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
# load weights
model_tfu.load_weights(dir_data+'models/weights_tfu.h5')
model_tfm.load_weights(dir_data+'models/weights_tfm.h5')

#### pytorch

In [13]:
# create net objects
model_ptu = PTUNET(n_features_u, n_filters, n_timesteps, n_linear, n_outputs)
model_ptm = PTMNET(n_features_m, n_filters, n_timesteps, n_linear, n_outputs)

In [14]:
# load weights
model_ptu.load_state_dict(torch.load(dir_data+'models/weights_ptu.pt'), strict=False)
model_ptm.load_state_dict(torch.load(dir_data+'models/weights_ptm.pt'), strict=False)

## Distribute Test data